In [3]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader
from langchain.schema import SystemMessage
import os

llm = ChatOpenAI(temperature=0.1)

# Wikipedia 검색 도구
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(description="위키피디아에서 검색할 쿼리")

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    위키피디아에서 정보를 검색할 때 사용하는 도구입니다.
    검색어를 입력하면 관련된 위키피디아 문서의 내용을 반환합니다.
    """
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaAPIWrapper()
        return wikipedia.run(query)

# DuckDuckGo 검색 도구
class WebSearchToolArgsSchema(BaseModel):
    query: str = Field(description="웹에서 검색할 쿼리")

class WebSearchTool(BaseTool):
    name = "WebSearchTool"
    description = """
    DuckDuckGo를 사용하여 웹에서 정보를 검색할 때 사용하는 도구입니다.
    검색어를 입력하면 관련된 웹 검색 결과를 반환합니다.
    """
    args_schema: Type[WebSearchToolArgsSchema] = WebSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

# 웹 스크래핑 도구
class WebScraperToolArgsSchema(BaseModel):
    url: str = Field(description="스크랩할 웹사이트의 URL")

class WebScraperTool(BaseTool):
    name = "WebScraperTool"
    description = """
    웹사이트의 내용을 스크랩하는 도구입니다.
    URL을 입력하면 해당 웹페이지의 텍스트 내용을 추출합니다.
    """
    args_schema: Type[WebScraperToolArgsSchema] = WebScraperToolArgsSchema

    def _run(self, url):
        loader = WebBaseLoader(url)
        docs = loader.load()
        return docs[0].page_content if docs else "추출 실패"

# txt 파일 저장 도구
class SaveToFileToolArgsSchema(BaseModel):
    content: str = Field(description="파일에 저장할 내용")
    filename: str = Field(description="저장할 파일 이름")

class SaveToFileTool(BaseTool):
    name = "SaveToFileTool"
    description = """
    텍스트 내용을 파일로 저장하는 도구입니다.
    내용과 파일 이름을 입력하면 .txt 파일로 저장합니다.
    """
    args_schema: Type[SaveToFileToolArgsSchema] = SaveToFileToolArgsSchema

    def _run(self, content: str, filename: str):
        if not filename.endswith('.txt'):
            filename += '.txt'
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"내용이 {filename} 파일에 저장되었습니다."

In [12]:
# 에이전트
agent = initialize_agent(
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "system_message": SystemMessage(content="""당신은 리서치를 수행하는 한국어 AI 어시스턴트입니다.
        사용자의 질문에 대해 다음 단계로 답변해주세요:
        1. Wikipedia와 DuckDuckGo에서 관련 정보를 검색
        2. 필요한 경우 웹사이트의 내용을 스크랩
        3. 수집한 정보를 종합하여 한국어로 답변
        4. 모든 조사 내용을 txt 파일로 저장
        
        답변은 항상 한국어로 해주시고, 전문적이고 객관적인 톤을 유지해주세요.
        조사결과를 모두 답변한 후에 조사결과를 저장한 txt파일의 이름과 저장경로를 출력해주세요."""
    )},
    tools=[
        WikipediaSearchTool(),
        WebSearchTool(),
        WebScraperTool(),
        SaveToFileTool()
    ],
)

In [13]:
query = "Research about the XZ backdoor"
result = agent.invoke(query)
print(result)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils